<div style="background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%); color: white; padding: 40px; margin: -10px -10px 20px -10px; border-radius: 0 0 15px 15px;">
<h1 style="margin: 0; font-size: 2.5em;">Packaging & Board Design</h1>
<p style="margin: 10px 0 0 0; font-size: 1.2em; opacity: 0.9;">Week 4, Session 1 — Fab Futures</p>
</div>

## 📋 Contents

1. [Chip Packaging Overview](#1-chip-packaging-overview)
2. [Package Types](#2-package-types)
3. [Wirebonding](#3-wirebonding)
4. [Evaluation Board Design](#4-evaluation-board-design)
5. [Testing Your Chip](#5-testing-your-chip)
6. [Debug Techniques](#6-debug-techniques)

In [ ]:
# Setup
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import FancyBboxPatch, Rectangle, Circle, Polygon
import numpy as np

print("✓ Setup complete!")

---
<a id="1-chip-packaging-overview"></a>
# 1. Chip Packaging Overview
---

### Why Package a Chip?

The silicon die is tiny and fragile. Packaging provides:

| Function | Description |
|----------|-------------|
| **Protection** | Mechanical, chemical, thermal protection |
| **Connectivity** | Route signals from tiny pads to solderable pins |
| **Heat dissipation** | Remove heat from the die |
| **Handling** | Makes the chip easy to place on a board |

### From Wafer to Package

```
Wafer → Dicing → Die Attach → Wirebond → Encapsulation → Test
```

In [ ]:
# Package cross-section
fig, ax = plt.subplots(figsize=(12, 6))

# Leadframe / substrate
ax.add_patch(Rectangle((0, 0), 12, 0.5, facecolor='#FFD700', edgecolor='black', linewidth=2))
ax.text(6, 0.25, 'Leadframe / Substrate', ha='center', va='center', fontsize=10)

# Die attach
ax.add_patch(Rectangle((4.5, 0.5), 3, 0.2, facecolor='#C0C0C0', edgecolor='black'))
ax.text(6, 0.6, 'Die attach', ha='center', va='center', fontsize=8)

# Die
ax.add_patch(Rectangle((4.5, 0.7), 3, 1.2, facecolor='#4169E1', edgecolor='black', linewidth=2))
ax.text(6, 1.3, 'Silicon Die', ha='center', va='center', fontsize=11, color='white', fontweight='bold')

# Bond pads on die
for i in range(5):
    x = 4.8 + i * 0.5
    ax.add_patch(Rectangle((x, 1.8), 0.3, 0.1, facecolor='gold', edgecolor='black'))

# Wirebonds
for i in range(5):
    x_die = 4.95 + i * 0.5
    x_lead = 1 + i * 2.2
    # Curved wirebond
    t = np.linspace(0, 1, 50)
    x = x_die + (x_lead - x_die) * t
    y = 1.85 + 1.5 * np.sin(np.pi * t)
    ax.plot(x, y, 'gold', linewidth=1.5)

# Lead fingers
for i in range(5):
    x = 1 + i * 2.2
    ax.add_patch(Rectangle((x - 0.2, 0), 0.4, 0.3, facecolor='#FFD700', edgecolor='black'))

# Mold compound outline
ax.add_patch(FancyBboxPatch((0.5, -0.2), 11, 4, boxstyle="round,pad=0.1",
             facecolor='none', edgecolor='#333', linewidth=2, linestyle='--'))
ax.text(10, 3.5, 'Mold\nCompound', ha='center', fontsize=9)

# Labels
ax.annotate('Bond wire', xy=(3, 2.5), fontsize=9,
           bbox=dict(boxstyle='round', facecolor='lightyellow'))
ax.annotate('Bond pad', xy=(5.5, 2.1), fontsize=8)

ax.set_xlim(-0.5, 12.5)
ax.set_ylim(-0.5, 4.5)
ax.set_aspect('equal')
ax.set_title('Package Cross-Section', fontsize=14, fontweight='bold')
ax.axis('off')

plt.tight_layout()
plt.show()

---
<a id="2-package-types"></a>
# 2. Package Types
---

### Through-Hole Packages (older, easier to prototype)

| Package | Pins | Notes |
|---------|------|-------|
| **DIP** (Dual In-line) | 8-64 | Breadboard-friendly! |
| **SIP** (Single In-line) | 4-24 | Memory modules |
| **PGA** (Pin Grid Array) | 100+ | Old CPUs |

### Surface Mount Packages (modern)

| Package | Pins | Notes |
|---------|------|-------|
| **SOIC** | 8-28 | Standard small outline |
| **QFP** (Quad Flat Pack) | 32-200+ | Leads on 4 sides |
| **QFN** (Quad Flat No-lead) | 8-100+ | No leads, just pads |
| **BGA** (Ball Grid Array) | 100-1000+ | Balls underneath |

### For Our Class

Educational tapeouts often use:
- **QFN** packages (small, easy to route)
- **Bare die** on carrier boards
- **Chip-on-board** (wirebond directly to PCB)

In [ ]:
# Package types visualization
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

# DIP
ax = axes[0]
ax.add_patch(Rectangle((1, 1), 3, 4, facecolor='#333', edgecolor='black', linewidth=2))
for i in range(8):
    ax.add_patch(Rectangle((0.5, 1.3 + i*0.4), 0.5, 0.15, facecolor='#C0C0C0', edgecolor='black'))
    ax.add_patch(Rectangle((4, 1.3 + i*0.4), 0.5, 0.15, facecolor='#C0C0C0', edgecolor='black'))
# Notch
ax.add_patch(Circle((2.5, 5), 0.2, facecolor='#333', edgecolor='white', linewidth=2))
ax.set_xlim(0, 5)
ax.set_ylim(0, 6)
ax.set_title('DIP-16', fontsize=11, fontweight='bold')
ax.set_aspect('equal')
ax.axis('off')

# QFP
ax = axes[1]
ax.add_patch(Rectangle((1, 1), 3, 3, facecolor='#333', edgecolor='black', linewidth=2))
for i in range(6):
    # Top/bottom
    ax.add_patch(Rectangle((1.3 + i*0.4, 4), 0.15, 0.4, facecolor='#C0C0C0', edgecolor='black'))
    ax.add_patch(Rectangle((1.3 + i*0.4, 0.6), 0.15, 0.4, facecolor='#C0C0C0', edgecolor='black'))
    # Left/right
    ax.add_patch(Rectangle((0.6, 1.3 + i*0.4), 0.4, 0.15, facecolor='#C0C0C0', edgecolor='black'))
    ax.add_patch(Rectangle((4, 1.3 + i*0.4), 0.4, 0.15, facecolor='#C0C0C0', edgecolor='black'))
ax.add_patch(Circle((1.3, 3.7), 0.15, facecolor='white'))
ax.set_xlim(0, 5)
ax.set_ylim(0, 5)
ax.set_title('QFP-48', fontsize=11, fontweight='bold')
ax.set_aspect('equal')
ax.axis('off')

# QFN
ax = axes[2]
ax.add_patch(Rectangle((1, 1), 3, 3, facecolor='#333', edgecolor='black', linewidth=2))
for i in range(5):
    ax.add_patch(Rectangle((1.4 + i*0.4, 0.95), 0.2, 0.3, facecolor='#C0C0C0', edgecolor='black'))
    ax.add_patch(Rectangle((1.4 + i*0.4, 3.75), 0.2, 0.3, facecolor='#C0C0C0', edgecolor='black'))
    ax.add_patch(Rectangle((0.95, 1.4 + i*0.4), 0.3, 0.2, facecolor='#C0C0C0', edgecolor='black'))
    ax.add_patch(Rectangle((3.75, 1.4 + i*0.4), 0.3, 0.2, facecolor='#C0C0C0', edgecolor='black'))
# Thermal pad
ax.add_patch(Rectangle((1.8, 1.8), 1.4, 1.4, facecolor='#C0C0C0', edgecolor='black'))
ax.add_patch(Circle((1.3, 3.5), 0.1, facecolor='white'))
ax.set_xlim(0, 5)
ax.set_ylim(0, 5)
ax.set_title('QFN-20', fontsize=11, fontweight='bold')
ax.set_aspect('equal')
ax.axis('off')

# BGA
ax = axes[3]
ax.add_patch(Rectangle((0.5, 0.5), 4, 4, facecolor='#333', edgecolor='black', linewidth=2))
for i in range(6):
    for j in range(6):
        ax.add_patch(Circle((1 + i*0.6, 1 + j*0.6), 0.15, facecolor='#C0C0C0', edgecolor='black'))
ax.add_patch(Circle((1, 3.5), 0.1, facecolor='white'))
ax.set_xlim(0, 5)
ax.set_ylim(0, 5)
ax.set_title('BGA-36', fontsize=11, fontweight='bold')
ax.set_aspect('equal')
ax.axis('off')

plt.tight_layout()
plt.show()

---
<a id="3-wirebonding"></a>
# 3. Wirebonding
---

**Wirebonding** connects the die pads to the package leads using thin gold or aluminum wires.

### Wirebond Diagram

For your design, you need to specify:
- Which die pad connects to which package pin
- Power (VDD) and Ground (GND) assignments
- Any special requirements (short wires for high-speed signals)

In [ ]:
# Wirebond diagram example
fig, ax = plt.subplots(figsize=(10, 10))

# Package outline
ax.add_patch(Rectangle((0, 0), 10, 10, facecolor='#F5F5F5', edgecolor='black', linewidth=2))

# Die
ax.add_patch(Rectangle((3, 3), 4, 4, facecolor='#4169E1', edgecolor='black', linewidth=2))
ax.text(5, 5, 'DIE', ha='center', va='center', fontsize=14, color='white', fontweight='bold')

# Package pins (simplified - 4 per side)
pin_labels = {
    'top': ['NC', 'TX', 'LED', 'NC'],
    'right': ['VDD', 'BTN', 'NC', 'GND'],
    'bottom': ['NC', 'CLK', 'RST', 'NC'],
    'left': ['GND', 'NC', 'NC', 'VDD']
}

colors = {'VDD': '#F44336', 'GND': '#2196F3', 'CLK': '#9C27B0', 
          'RST': '#FF9800', 'TX': '#4CAF50', 'BTN': '#FFC107', 
          'LED': '#E91E63', 'NC': '#9E9E9E'}

# Top pins
for i, label in enumerate(pin_labels['top']):
    x = 2 + i * 2
    ax.add_patch(Rectangle((x - 0.3, 9.5), 0.6, 0.4, facecolor=colors[label], edgecolor='black'))
    ax.text(x, 10.2, label, ha='center', fontsize=8)
    if label != 'NC':
        ax.plot([x, x], [9.5, 7], colors[label], linewidth=1.5)

# Bottom pins
for i, label in enumerate(pin_labels['bottom']):
    x = 2 + i * 2
    ax.add_patch(Rectangle((x - 0.3, 0.1), 0.6, 0.4, facecolor=colors[label], edgecolor='black'))
    ax.text(x, -0.4, label, ha='center', fontsize=8)
    if label != 'NC':
        ax.plot([x, x], [0.5, 3], colors[label], linewidth=1.5)

# Left pins
for i, label in enumerate(pin_labels['left']):
    y = 2 + i * 2
    ax.add_patch(Rectangle((0.1, y - 0.3), 0.4, 0.6, facecolor=colors[label], edgecolor='black'))
    ax.text(-0.4, y, label, ha='center', va='center', fontsize=8, rotation=90)
    if label != 'NC':
        ax.plot([0.5, 3], [y, y], colors[label], linewidth=1.5)

# Right pins
for i, label in enumerate(pin_labels['right']):
    y = 2 + i * 2
    ax.add_patch(Rectangle((9.5, y - 0.3), 0.4, 0.6, facecolor=colors[label], edgecolor='black'))
    ax.text(10.4, y, label, ha='center', va='center', fontsize=8, rotation=90)
    if label != 'NC':
        ax.plot([9.5, 7], [y, y], colors[label], linewidth=1.5)

# Die pads (simplified)
die_pads = [
    (3.2, 6.8, 'TX'), (5, 6.8, 'LED'),
    (6.8, 5.5, 'VDD'), (6.8, 4.5, 'BTN'),
    (6.8, 3.5, 'GND'),
    (5, 3.2, 'CLK'), (4, 3.2, 'RST'),
    (3.2, 4.5, 'GND'), (3.2, 5.5, 'VDD'),
]

for x, y, label in die_pads:
    ax.add_patch(Rectangle((x - 0.15, y - 0.15), 0.3, 0.3, 
                           facecolor='gold', edgecolor='black'))

ax.set_xlim(-1, 11)
ax.set_ylim(-1, 11)
ax.set_aspect('equal')
ax.set_title('Wirebond Diagram Example', fontsize=14, fontweight='bold')
ax.axis('off')

# Legend
ax.text(8, -0.8, 'VDD=Power  GND=Ground  NC=No Connect', fontsize=8)

plt.tight_layout()
plt.show()

---
<a id="4-evaluation-board-design"></a>
# 4. Evaluation Board Design
---

An **evaluation board** (eval board) lets you test your chip by providing:

| Feature | Purpose |
|---------|--------|
| **Power supply** | Clean VDD and GND |
| **Decoupling capacitors** | Filter power noise |
| **Clock source** | Crystal or oscillator |
| **Reset circuit** | Power-on reset |
| **I/O connectors** | Access to chip pins |
| **Debug headers** | Logic analyzer, JTAG |

### Typical Eval Board Components

```
┌─────────────────────────────────────────┐
│  ┌─────┐                     ┌────────┐ │
│  │Power│    ┌─────────┐      │USB-UART│ │
│  │ In  │    │         │      │ Bridge │ │
│  └──┬──┘    │  YOUR   │      └───┬────┘ │
│     │       │  CHIP   │          │      │
│  ┌──┴──┐    │         │      ┌───┴───┐  │
│  │Vreg │    └────┬────┘      │Headers│  │
│  └─────┘         │           └───────┘  │
│           ┌──────┼──────┐               │
│           │Decap │ Xtal │               │
│           └──────┴──────┘               │
└─────────────────────────────────────────┘
```

### KiCad for PCB Design

[KiCad](https://www.kicad.org/) is a free, open-source PCB design tool.

**Basic workflow:**
1. **Schematic** - Draw circuit connections
2. **Footprints** - Assign physical package shapes
3. **Layout** - Place components and route traces
4. **Gerbers** - Export manufacturing files

**PCB fabs for prototypes:**
- JLCPCB (~$2 for 5 boards)
- PCBWay
- OSH Park (purple boards!)

---
<a id="5-testing-your-chip"></a>
# 5. Testing Your Chip
---

### Test Setup

| Equipment | Purpose |
|-----------|--------|
| **Power supply** | Provide VDD (often 1.8V or 3.3V) |
| **Multimeter** | Measure voltage, current |
| **Oscilloscope** | See digital signals |
| **Logic analyzer** | Capture many digital signals |
| **USB-UART adapter** | Serial communication with laptop |

### Test Plan Template

1. **Power-up test**
   - Does the chip draw expected current?
   - No smoke? (seriously, check this first!)

2. **Basic functionality**
   - Does reset work?
   - Can you see clock activity?
   - Do outputs toggle?

3. **Feature tests**
   - Test each feature of your design
   - Compare to simulation results

### For Our Projects

| Project | Test Method |
|---------|-------------|
| **Fortune Teller** | Press button, see output on serial terminal |
| **Pocket Synth** | Press keys, listen to speaker |
| **Dice Roller** | Press button, see 7-segment display |
| **LED Messenger** | Power up, see LEDs animate |

---
<a id="6-debug-techniques"></a>
# 6. Debug Techniques
---

### When Things Don't Work

| Symptom | Check |
|---------|-------|
| **No power** | Solder joints, voltage at pins |
| **High current** | Short circuit, wrong voltage |
| **No activity** | Clock, reset, enable signals |
| **Wrong output** | Input values, state machine |
| **Glitchy output** | Power supply noise, timing |

### Debug Strategies

1. **Divide and conquer**
   - Isolate which block is failing
   - Test inputs and outputs of each block

2. **Compare to simulation**
   - Apply same inputs as testbench
   - Do outputs match?

3. **Add observability**
   - Route internal signals to spare pins
   - Add debug registers (if you planned ahead!)

### Design for Debug (DFD)

Plan ahead in your design:
- **Spare I/O pins** for probing internal signals
- **Bypass modes** to isolate blocks
- **Status registers** readable via serial

---
<div style="background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%); color: white; padding: 30px; margin: 20px -10px -10px -10px; border-radius: 15px 15px 0 0; text-align: center;">

## Summary

- **Packaging** protects the die and provides connectivity
- **Wirebonding** connects die pads to package pins
- **Eval boards** provide power, clock, and I/O access
- **Testing** verifies your chip works in silicon
- **Debug** requires planning and systematic approach

### Homework
1. Run final DRC/LVS on your design
2. Document your pin assignments
3. Create a test plan
4. Prepare your presentation for Thursday!

</div>